In [1]:
import os
import rasterio
import geopandas as gpd
from rasterio.mask import mask

mosaics_dir = "./Data/Images/Mosaics"
patches_dir = "./Data/Images/Patches"
grid_path = "./Data/Grid/Grid.shp"
grid = gpd.read_file(grid_path)
walkthrough = os.walk(mosaics_dir)

for root, folders, files in walkthrough:
  if files:
    print(f"[DEBUG] Current file path: {os.path.join(root, files[0])}")
    
    with rasterio.open(os.path.join(root, files[0])) as src:
      rst = src.read()
      print(f"        Making patches [", end="")

      for i, patch in grid.to_crs(src.crs).iterrows():
        if not (i + 1) % (len(grid) // 100): print(".", end="")

        out_image, out_transform = mask(src, [patch.geometry], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
          "height": out_image.shape[1],
          "width": out_image.shape[2],
          "transform": out_transform
        })

        output_folder = os.path.join(patches_dir, root[-8:])
        output_file = f"{output_folder}/Patch_{int(patch.id)}.tif"
        os.makedirs(output_folder, exist_ok=True)

        with rasterio.open(output_file, "w", **out_meta) as dst:
          dst.write(out_image)

      print("]")

[DEBUG] Current file path: ./Data/Images/Mosaics\20230101\Mosaic.tif
        Making patches [....................................................................................................]
[DEBUG] Current file path: ./Data/Images/Mosaics\20230117\Mosaic.tif
        Making patches [....................................................................................................]
[DEBUG] Current file path: ./Data/Images/Mosaics\20230202\Mosaic.tif
        Making patches [....................................................................................................]
[DEBUG] Current file path: ./Data/Images/Mosaics\20230218\Mosaic.tif
        Making patches [....................................................................................................]
[DEBUG] Current file path: ./Data/Images/Mosaics\20230306\Mosaic.tif
        Making patches [....................................................................................................]
[DEBUG] Current file path